# BASE PARAM SILVER NOTEBOOK v1.0.0

## Parameters

In [ ]:
# FABRICENGINEER
FABRICENGINEER_PACKAGE_VERSION = "1.0.1"
FABRICENGINEER_SILVER_STRATEGY = "transform.silver.insertonly"

# HISTORIZE TABLE
HISTORIZE = None

# SOURCE CONFIG
SOURCE_LAKEHOUSE = None
SOURCE_SCHEMA = None
SOURCE_TABLENAME = None

# DESTINATION CONFIG
DESTINATION_LAKEHOUSE = None
DESTINATION_SCHEMA = None
DESTINATION_TABLENAME = None

# NATURAL KEY COLUMNS
NK_COLUMNS = None

# EXTRA CONSTANT COLUMNS
CONSTANT_COLUMNS = [
    # ConstantColumn(name="WAVE_MANDANT", value="VTSD", part_of_nk=True)
]

# DELTA LOAD CONFIG
IS_DELTA_LOAD = None
DELTA_LOAD_USE_BROADCAST = None

# INCLUDE / EXCLUDES COLUMNS FROM COMPARING
EXCLUDE_COLUMNS_FROM_COMPARING = []
INCLUDE_COLUMNS_AT_COMPARING = []  # when is present (len >= 1), the exclude columns will ignored

# PARTITION BY COLUMNS
PARTITION_BY_COLUMNS = []

StatementMeta(, 014180a5-345f-499b-8ce9-d4bd97ff1d4b, 3, Finished, Available, Finished)

In [ ]:
# Bring parameters in shape

if isinstance(NK_COLUMNS, str):
    NK_COLUMNS = list([column.strip() for column in NK_COLUMNS.split(",")])
    print("NK-COLUMNS: ", NK_COLUMNS)

if isinstance(EXCLUDE_COLUMNS_FROM_COMPARING, str):
    EXCLUDE_COLUMNS_FROM_COMPARING = list(set([column.strip() for column in EXCLUDE_COLUMNS_FROM_COMPARING.split(",")]))
    print("EXCLUDE_COLUMNS_FROM_COMPARING: ", EXCLUDE_COLUMNS_FROM_COMPARING)

if isinstance(INCLUDE_COLUMNS_AT_COMPARING, str):
    INCLUDE_COLUMNS_AT_COMPARING = list(set([column.strip() for column in INCLUDE_COLUMNS_AT_COMPARING.split(",")]))
    print("INCLUDE_COLUMNS_AT_COMPARING: ", INCLUDE_COLUMNS_AT_COMPARING)

if isinstance(PARTITION_BY_COLUMNS, str):
    PARTITION_BY_COLUMNS = list([column.strip() for column in PARTITION_BY_COLUMNS.split(",")])
    print("PARTITION_BY_COLUMNS: ", PARTITION_BY_COLUMNS)


DESTINATION_TABLENAME = DESTINATION_TABLENAME.lower()

## Module Imports

In [ ]:
%run IMPORT FABRICENGINEER PACKAGE

StatementMeta(, 014180a5-345f-499b-8ce9-d4bd97ff1d4b, 42, Finished, Available, Finished)

Passed functions [<function get_module_code at 0x73f0ae57b060>]


In [ ]:
strategy_module_code = get_module_code(FABRICENGINEER_SILVER_STRATEGY, FABRICENGINEER_PACKAGE_VERSION)

assert isinstance(strategy_module_code, str)

exec(strategy_module_code, globals())

assert (
    "SilverIngestionInsertOnlyService" in str(etl) or
    "SilverIngestionSCD2Service" in str(etl)
)
assert "LakehouseTable" in str(LakehouseTable)
assert "ConstantColumn" in str(ConstantColumn)
assert "TimeLogger" in str(timer)

StatementMeta(, 014180a5-345f-499b-8ce9-d4bd97ff1d4b, 49, Submitted, Running, Running)

Fetch code from notebook.


## Configure ETL

In [ ]:
source_table = LakehouseTable(
    lakehouse=SOURCE_LAKEHOUSE,
    schema=SOURCE_SCHEMA,
    table=SOURCE_TABLENAME
)

destination_table = LakehouseTable(
    lakehouse=DESTINATION_LAKEHOUSE,
    schema=DESTINATION_SCHEMA,
    table=DESTINATION_TABLENAME
)

print(source_table)
print(destination_table)

In [ ]:
transformations: dict = {
    "{tableName}": lambda: print("Add transformation function (instead of lambda: ...)"),
}

In [ ]:
etl.init(
    spark_=spark,
    notebookutils_=notebookutils,
    source_table=source_table,
    destination_table=destination_table,
    nk_columns=NK_COLUMNS,
    constant_columns=CONSTANT_COLUMNS,
    is_delta_load=IS_DELTA_LOAD,
    delta_load_use_broadcast=DELTA_LOAD_USE_BROADCAST,
    transformations=transformations,
    exclude_comparing_columns=EXCLUDE_COLUMNS_FROM_COMPARING,
    include_comparing_columns=INCLUDE_COLUMNS_AT_COMPARING,
    historize=HISTORIZE,
    partition_by_columns=PARTITION_BY_COLUMNS,
    df_bronze=None
)

print(etl)

## Run ETL

In [ ]:
timer.start().log()
etl.run()
timer.stop().log()